pip install numpy  
pip install pandas  
pip install openpyxl  
pip install tqdm
pip install ffmpeg  
pip install pydub  
conda install -c conda-forge librosa  
pip install -U praat-parselmouth

In [1]:
import pydub
import pandas as pd
from tqdm import tqdm
import os

pydub.AudioSegment.converter = r"C:/Users/Kinza/anaconda3/envs/pie/Library/bin/ffmpeg.exe" #CHANGE THIS

# Data loading

In [2]:
def get_end_from_start(df_mail):
    end             = df_mail.iloc[1:,:]['start'].copy()
    end[len(end)+1] = -1
    df_mail['end']  = end.values
    return df_mail

def get_start_end_from_file(file):
    #Get start and end times
    df = pd.read_excel(file)
    df = df.rename(columns={'time':'start'}) 
    df['start'] = df['start']*1000 #in ms

    df = df.groupby('mail').apply(get_end_from_start)
    return df

def load_audios(video_folder,startend_file) :
    '''Loads the audios from each interview question.
    Arguments:
        video_folder : str. The name of the folder containing mp4 videos.
        startend_file : str. The name of the file containing the video informations. 
            Must contain columns 'mail' and 'time'.
    Returns a list of audios
    '''
    filenames = tqdm(os.listdir(video_folder))
    df_startend = get_start_end_from_file(startend_file)
    audios = list(map(lambda f : load_audio(video_folder,df_startend,f), filenames))
    return [item for sublist in audios for item in sublist]


In [3]:
from audio_feats_extract import load_audio

video_folder = 'videos/'
df_name = 'test.xlsx'

audios = load_audios(video_folder,df_name)

  0%|          | 0/3 [00:00<?, ?it/s]C:\Users\Kinza\anaconda3\envs\pie\lib\site-packages\librosa\core\convert.py:1350: RuntimeWarning: divide by zero encountered in log10
  + 2 * np.log10(f_sq)
 33%|███▎      | 1/3 [02:25<04:50, 145.29s/it]


KeyboardInterrupt: 

In [ ]:
print(audios[1].spectral_features)
print(audios[1].pauses_features)
print(audios[1].prosodic_features)
print(audios[1].self.speech_features)

,rms,spec_cent,spec_bw,rolloff,zcr,mfcc0,mfcc1,mfcc2,mfcc3,mfcc4,...,chroma_stft2,chroma_stft3,chroma_stft4,chroma_stft5,chroma_stft6,chroma_stft7,chroma_stft8,chroma_stft9,chroma_stft10,chroma_stft11
mean,0.044871,2839.038942,3142.372696,5024.803844,0.069704,-396.626747,99.945612,14.077295,15.189622,15.137396,...,0.257474,0.193744,0.160091,0.163266,0.138997,0.165495,0.250408,0.344625,0.467093,0.483843
std,0.037229,2605.151269,1319.556400,4997.968679,0.095941,73.999082,62.420127,35.329366,34.547575,25.073616,...,0.308087,0.298261,0.240579,0.262583,0.218006,0.230251,0.301918,0.347450,0.360368,0.324898
min,0.000089,471.888457,1241.525787,234.375000,0.007324,-553.980164,-113.828293,-86.339127,-93.917351,-80.448517,...,0.000176,0.000218,0.000204,0.000123,0.000108,0.000145,0.000114,0.000248,0.000658,0.001484
max,0.352426,12135.280498,6781.699960,16898.437500,0.528809,-151.442795,217.433685,94.427536,122.327248,85.647621,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


# Load features

In [ ]:
import pandas as pd

all_features = []
for (i,audio) in enumerate(audios):
    features = pd.concat([audio.pauses_features,audio.spectral_features,audio.prosodic_features])
    features['email'] = audio.email
    features['question'] = audio.question

    features['temp'] = features.index
    features = features.pivot_table(index='timestamp', columns='temp', values=features.columns[-3], aggfunc='first')

    all_features = all_features.append(features)

all_features = pd.concat(all_features)

# PCA

In [ ]:
from pca import pca

model = pca(n_components=2)

# Fit transform
features_pca = model.fit_transform(features.drop('classes',axis=1))

print(model.compute_topfeat())

# Scatter first 2 PCs
fig, ax = model.scatter(legend=False)

# Make biplot with the number of features
fig, ax = model.biplot(n_feat=10,legend=False)

import matplotlib.pyplot as plt
plt.scatter(features_pca['PC']['PC1'],features_pca['PC']['PC2'],c=features['classes'])